Import libraries and dataset


In [ ]:
import numpy as np
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization, AlphaDropout
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
print(y_train.flatten())

[6 9 9 ... 9 1 1]


In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

Preprocess the data


In [ ]:
x_train=x_train.astype('float32')/255.0
#y_train=y_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0
#y_test=y_test.astype('floa32')/255.0

convert output to multiclass and binaryclass

In [ ]:
y_train.shape

(50000, 1)

In [ ]:
y_train_multiclass=y_train
y_test_multiclass=y_test

In [ ]:
# Binary Class
animal_classes = [2,3,4,5,6]
vehicle_classes = [0,1,7,8,9]

# Convert to binary labels
y_train_binaryclass = np.where(np.isin(y_train, animal_classes), 0, 1)
y_test_binaryclass = np.where(np.isin(y_test, animal_classes), 0, 1)

In [ ]:
y_train_multiclass.shape

(50000, 1)

In [ ]:
y_train_binaryclass.shape

(50000, 1)

Train the model

In [ ]:
# Build model
inputs = Input(shape=(32, 32, 3))
conv1 = Conv2D(64, (3, 3), activation='relu')(inputs)
maxpool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(128, (3, 3), activation='relu')(maxpool1)
maxpool2 = MaxPooling2D((2, 2))(conv2)
flatten = Flatten()(maxpool2)

Shared_hiddenlayer=Dense(128, activation='relu')(flatten)


output_multiclass = Dense(10, activation='softmax', name='multi_class')(Shared_hiddenlayer)

# Binary classification output
output_binaryclass = Dense(1, activation='sigmoid', name='binary_class')(Shared_hiddenlayer)

In [ ]:
model=Model(inputs=inputs,outputs=[output_multiclass,output_binaryclass])


model.compile(optimizer='Adam',
              loss={'multi_class':'categorical_crossentropy',
                    'binary_class':'binary_crossentropy'},
              metrics={'multi_class':'accuracy',
                       'binary_class':'accuracy'})

In [ ]:
#Train the model
model.fit(x_train,{'multi_class':y_train_multiclass,'binary_class':y_train_binaryclass},
          validation_data=(x_test,{'multi_class':y_test_multiclass,'binary_class':y_test_binaryclass}),epochs=10,batch_size=64)

Epoch 1/10
782/782 [==============================] - 197s 249ms/step - loss: 0.0181 - multi_class_loss: 0.0129 - binary_class_loss: 0.0052 - multi_class_accuracy: 0.9987 - binary_class_accuracy: 0.9995 - val_loss: 3634.1265 - val_multi_class_loss: 2657.8198 - val_binary_class_loss: 976.3055 - val_multi_class_accuracy: 0.1000 - val_binary_class_accuracy: 0.5000
Epoch 2/10
782/782 [==============================] - 180s 230ms/step - loss: 2.5085e-04 - multi_class_loss: 1.2259e-04 - binary_class_loss: 1.2826e-04 - multi_class_accuracy: 1.0000 - binary_class_accuracy: 1.0000 - val_loss: 4009.2932 - val_multi_class_loss: 2918.8455 - val_binary_class_loss: 1090.4478 - val_multi_class_accuracy: 0.1000 - val_binary_class_accuracy: 0.5000
Epoch 3/10
782/782 [==============================] - 181s 232ms/step - loss: 1.4455e-04 - multi_class_loss: 7.2782e-05 - binary_class_loss: 7.1771e-05 - multi_class_accuracy: 1.0000 - binary_class_accuracy: 1.0000 - val_loss: 4233.6729 - val_multi_class_loss

Evaluate the model performance

In [ ]:
loss, multi_class_loss, binary_class_loss, multi_class_accuracy, binary_class_accuracy = model.evaluate(x_test,
                                                                                                        {'multi_class': y_test_multiclass,
                                                                                                         'binary_class': y_test_binaryclass})

313/313 [==============================] - 10s 30ms/step - loss: 4926.0381 - multi_class_loss: 3557.4780 - binary_class_loss: 1368.5598 - multi_class_accuracy: 0.1000 - binary_class_accuracy: 0.5000


Tune Hyperparameters, introduce dropout, learning rate, batch normalization

In [ ]:
inputs = Input(shape=(32, 32, 3))

conv1 = Conv2D(64, (3, 3), activation='relu')(inputs)
batchnorm1 = BatchNormalization()(conv1)
maxpool1 = MaxPooling2D((2, 2))(batchnorm1)


conv2 = Conv2D(128, (3, 3), activation='relu')(maxpool1)
batchnorm2 = BatchNormalization()(conv2)
maxpool2 = MaxPooling2D((2, 2))(batchnorm2)



flatten = Flatten()(maxpool2)

# Shared hidden layer
dense1 = Dense(128, activation='relu')(flatten)
batchnorm4 = BatchNormalization()(dense1)
dropout1 = Dropout(0.3)(batchnorm4)

# Multi-class classification output
output_multiclass = Dense(10, activation='softmax', name='multi_class')(dropout1)

# Binary classification output
output_binaryclass = Dense(1, activation='sigmoid', name='binary_class')(dropout1)


model = Model(inputs=inputs, outputs=[output_multiclass, output_binaryclass])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'multi_class': 'categorical_crossentropy', 'binary_class': 'binary_crossentropy'},
    metrics={'multi_class': 'accuracy', 'binary_class': 'accuracy'}
)

In [ ]:
loss, multi_class_loss, binary_class_loss, multi_class_accuracy, binary_class_accuracy = model.evaluate(x_test,
                                                                                                        {'multi_class': y_test_multiclass,
                                                                                                         'binary_class': y_test_binaryclass})

313/313 [==============================] - 11s 34ms/step - loss: 3796.7871 - multi_class_loss: 2966.4885 - binary_class_loss: 830.2988 - multi_class_accuracy: 0.1000 - binary_class_accuracy: 0.5000


Performance tuning with SGD optimizer and Sparse-Catagorical loss function

In [ ]:
inputs = Input(shape=(32, 32, 3))

conv1 = Conv2D(64, (3, 3), activation='relu')(inputs)
batchnorm1 = BatchNormalization()(conv1)
maxpool1 = MaxPooling2D((2, 2))(batchnorm1)

conv2 = Conv2D(128, (3, 3), activation='relu')(maxpool1)
batchnorm2 = BatchNormalization()(conv2)
maxpool2 = MaxPooling2D((2, 2))(batchnorm2)


flatten = Flatten()(maxpool2)

# Shared hidden layer
dense1 = Dense(128, activation='relu')(flatten)
batchnorm4 = BatchNormalization()(dense1)
dropout1 = Dropout(0.3)(batchnorm4)  

# Multi-class classification output
output_multiclass = Dense(10, activation='softmax', name='multi_class')(dropout1)

# Binary classification output
output_binaryclass = Dense(1, activation='sigmoid', name='binary_class')(dropout1)

model = Model(inputs=inputs, outputs=[output_multiclass, output_binaryclass])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'multi_class': 'sparse_categorical_crossentropy', 'binary_class': 'binary_crossentropy'},
    metrics={'multi_class': 'accuracy', 'binary_class': 'accuracy'}
)

In [ ]:
#Train the model
model.fit(x_train,{'multi_class':y_train_multiclass,'binary_class':y_train_binaryclass},
          validation_data=(x_test,{'multi_class':y_test_multiclass,'binary_class':y_test_binaryclass}),epochs=3,batch_size=64)

Epoch 1/3
782/782 [==============================] - 101s 126ms/step - loss: 0.0814 - multi_class_loss: 0.0595 - binary_class_loss: 0.0219 - multi_class_accuracy: 0.0984 - binary_class_accuracy: 0.9920 - val_loss: 3276.6787 - val_multi_class_loss: 2613.4626 - val_binary_class_loss: 663.2162 - val_multi_class_accuracy: 0.1000 - val_binary_class_accuracy: 0.5000
Epoch 2/3
782/782 [==============================] - 94s 121ms/step - loss: 0.0011 - multi_class_loss: 4.7579e-04 - binary_class_loss: 6.2224e-04 - multi_class_accuracy: 0.1000 - binary_class_accuracy: 1.0000 - val_loss: 3094.2341 - val_multi_class_loss: 2463.8433 - val_binary_class_loss: 630.3903 - val_multi_class_accuracy: 0.1000 - val_binary_class_accuracy: 0.5000
Epoch 3/3
782/782 [==============================] - 94s 120ms/step - loss: 6.0401e-04 - multi_class_loss: 2.6488e-04 - binary_class_loss: 3.3913e-04 - multi_class_accuracy: 0.1000 - binary_class_accuracy: 1.0000 - val_loss: 3095.2354 - val_multi_class_loss: 2439.678

In [ ]:
loss, multi_class_loss, binary_class_loss, multi_class_accuracy, binary_class_accuracy = model.evaluate(x_test,
                                                                                                        {'multi_class': y_test_multiclass,
                                                                                                         'binary_class': y_test_binaryclass})

313/313 [==============================] - 6s 19ms/step - loss: 3095.2363 - multi_class_loss: 2439.6777 - binary_class_loss: 655.5584 - multi_class_accuracy: 0.1000 - binary_class_accuracy: 0.5000


Apply L2 regularization

In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal

In [ ]:
# Input layer
inputs = Input(shape=(32, 32, 3))

conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001))(inputs)
batchnorm1 = BatchNormalization()(conv1)
maxpool1 = MaxPooling2D((2, 2))(batchnorm1)

conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001))(maxpool1)
batchnorm2 = BatchNormalization()(conv2)
maxpool2 = MaxPooling2D((2, 2))(batchnorm2)


flatten = Flatten()(maxpool2)

# Shared hidden layer with he_normal initialization and L2 regularization
dense1 = Dense(128, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001))(flatten)
batchnorm4 = BatchNormalization()(dense1)
dropout1 = Dropout(0.3)(batchnorm4)


# Multi-class classification output
output_multiclass = Dense(10, activation='softmax', name='multi_class')(dropout1)

# Binary classification output
output_binaryclass = Dense(1, activation='sigmoid', name='binary_class')(dropout1)

model = Model(inputs=inputs, outputs=[output_multiclass, output_binaryclass])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'multi_class': 'sparse_categorical_crossentropy', 'binary_class': 'binary_crossentropy'},
    metrics={'multi_class': 'accuracy', 'binary_class': 'accuracy'}
)

In [ ]:
model.fit(x_train,{'multi_class':y_train_multiclass,'binary_class':y_train_binaryclass},
          validation_data=(x_test,{'multi_class':y_test_multiclass,'binary_class':y_test_binaryclass}),epochs=3,batch_size=64)

Epoch 1/3
782/782 [==============================] - 235s 296ms/step - loss: 0.8839 - multi_class_loss: 0.3485 - binary_class_loss: 0.1312 - multi_class_accuracy: 0.0903 - binary_class_accuracy: 0.9469 - val_loss: 993.9958 - val_multi_class_loss: 769.1321 - val_binary_class_loss: 224.6839 - val_multi_class_accuracy: 0.1009 - val_binary_class_accuracy: 0.5000
Epoch 2/3
782/782 [==============================] - 226s 289ms/step - loss: 0.0989 - multi_class_loss: 0.0033 - binary_class_loss: 0.0025 - multi_class_accuracy: 0.0999 - binary_class_accuracy: 0.9999 - val_loss: 912.1151 - val_multi_class_loss: 746.6787 - val_binary_class_loss: 165.3956 - val_multi_class_accuracy: 0.1000 - val_binary_class_accuracy: 0.5000
Epoch 3/3
782/782 [==============================] - 226s 289ms/step - loss: 0.0284 - multi_class_loss: 0.0011 - binary_class_loss: 7.8850e-04 - multi_class_accuracy: 0.1000 - binary_class_accuracy: 1.0000 - val_loss: 324.5755 - val_multi_class_loss: 191.0323 - val_binary_class

In [ ]:
loss, multi_class_loss, binary_class_loss, multi_class_accuracy, binary_class_accuracy = model.evaluate(x_test,
                                                                                                        {'multi_class': y_test_multiclass,
                                                                                                         'binary_class': y_test_binaryclass})

In [ ]:
# Input layer
inputs = Input(shape=(32, 32, 3))

conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.01))(inputs)
batchnorm1 = BatchNormalization()(conv1)
alpha_dropout1 = AlphaDropout(0.5)(batchnorm1)  # AlphaDropout with a dropout rate of 0.5
maxpool1 = MaxPooling2D((2, 2))(alpha_dropout1)

conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.01))(maxpool1)
batchnorm2 = BatchNormalization()(conv2)
alpha_dropout2 = AlphaDropout(0.5)(batchnorm2)  # AlphaDropout with a dropout rate of 0.5
maxpool2 = MaxPooling2D((2, 2))(alpha_dropout2)

conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.01))(maxpool2)
batchnorm3 = BatchNormalization()(conv3)
alpha_dropout3 = AlphaDropout(0.5)(batchnorm3)  # AlphaDropout with a dropout rate of 0.5
maxpool3 = MaxPooling2D((2, 2))(alpha_dropout3)

flatten = Flatten()(maxpool3)

# Shared hidden layer with he_normal initialization and L2 regularization
dense1 = Dense(128, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.01))(flatten)
batchnorm4 = BatchNormalization()(dense1)
dropout1 = Dropout(0.3)(batchnorm4)


# Multi-class classification output
output_multiclass = Dense(10, activation='softmax', name='multi_class')(dropout1)

# Binary classification output
output_binaryclass = Dense(1, activation='sigmoid', name='binary_class')(dropout1)

# Create the model
model = Model(inputs=inputs, outputs=[output_multiclass, output_binaryclass])

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss={'multi_class': 'sparse_categorical_crossentropy', 'binary_class': 'binary_crossentropy'},
    metrics={'multi_class': 'accuracy', 'binary_class': 'accuracy'}
)

In [ ]:
model.fit(x_train,{'multi_class':y_train_multiclass,'binary_class':y_train_binaryclass},
          validation_data=(x_test,{'multi_class':y_test_multiclass,'binary_class':y_test_binaryclass}),epochs=2,batch_size=64)

Epoch 1/2
782/782 [==============================] - 144s 174ms/step - loss: 0.5364 - multi_class_loss: 0.0356 - binary_class_loss: 0.0135 - multi_class_accuracy: 0.0991 - binary_class_accuracy: 0.9947 - val_loss: 18.0920 - val_multi_class_loss: 12.9125 - val_binary_class_loss: 5.1794 - val_multi_class_accuracy: 0.1000 - val_binary_class_accuracy: 0.5000
Epoch 2/2
782/782 [==============================] - 134s 171ms/step - loss: 3.7140e-05 - multi_class_loss: 4.4966e-06 - binary_class_loss: 2.3514e-05 - multi_class_accuracy: 0.1000 - binary_class_accuracy: 1.0000 - val_loss: 19.4016 - val_multi_class_loss: 13.7099 - val_binary_class_loss: 5.6917 - val_multi_class_accuracy: 0.1000 - val_binary_class_accuracy: 0.5000


In [ ]:
loss, multi_class_loss, binary_class_loss, multi_class_accuracy, binary_class_accuracy = model.evaluate(x_train,
                                                                                                        {'multi_class': y_train_multiclass,
                                                                                                         'binary_class': y_train_binaryclass})

1563/1563 [==============================] - 32s 20ms/step - loss: 1.7653e-05 - multi_class_loss: 2.2650e-06 - binary_class_loss: 1.1383e-05 - multi_class_accuracy: 0.1000 - binary_class_accuracy: 1.0000


Max-norm regularization

In [ ]:
from tensorflow.keras.constraints import max_norm

In [ ]:


# Input layer
inputs = Input(shape=(32, 32, 3))

conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=he_normal(),
               kernel_regularizer=l2(0.001), kernel_constraint=max_norm(3))(inputs)
batchnorm1 = BatchNormalization()(conv1)
alpha_dropout1 = AlphaDropout(0.3)(batchnorm1)  # Reduced dropout rate
maxpool1 = MaxPooling2D((2, 2))(alpha_dropout1)

conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=he_normal(),
               kernel_regularizer=l2(0.001), kernel_constraint=max_norm(3))(maxpool1)
batchnorm2 = BatchNormalization()(conv2)
alpha_dropout2 = AlphaDropout(0.3)(batchnorm2)  # Reduced dropout rate
maxpool2 = MaxPooling2D((2, 2))(alpha_dropout2)

# Third convolutional layer with 256 filters, he_normal initialization, L2 regularization, AlphaDropout, and max-norm constraint
conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=he_normal(),
               kernel_regularizer=l2(0.001), kernel_constraint=max_norm(3))(maxpool2)
batchnorm3 = BatchNormalization()(conv3)
alpha_dropout3 = AlphaDropout(0.3)(batchnorm3)  # Reduced dropout rate
maxpool3 = MaxPooling2D((2, 2))(alpha_dropout3)

flatten = Flatten()(maxpool3)

dense1 = Dense(128, activation='relu', kernel_initializer=he_normal(),
               kernel_regularizer=l2(0.001), kernel_constraint=max_norm(3))(flatten)
batchnorm4 = BatchNormalization()(dense1)
dropout1 = Dropout(0.3)(batchnorm4)

# Multi-class classification output
output_multiclass = Dense(10, activation='softmax', name='multi_class')(dropout1)

# Binary classification output
output_binaryclass = Dense(1, activation='sigmoid', name='binary_class')(dropout1)

model = Model(inputs=inputs, outputs=[output_multiclass, output_binaryclass])




In [ ]:
# Compile the model with an increased learning rate
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss={'multi_class': 'sparse_categorical_crossentropy', 'binary_class': 'binary_crossentropy'},
    metrics={'multi_class': 'accuracy', 'binary_class': 'accuracy'}
)

In [ ]:
model.fit(x_train,{'multi_class':y_train_multiclass,'binary_class':y_train_binaryclass},
          validation_data=(x_test,{'multi_class':y_test_multiclass,'binary_class':y_test_binaryclass}),epochs=10,batch_size=64)

Epoch 1/10
782/782 [==============================] - 136s 174ms/step - loss: 2.2634 - multi_class_loss: 1.4898 - binary_class_loss: 0.3735 - multi_class_accuracy: 0.4575 - binary_class_accuracy: 0.8386 - val_loss: 898.2033 - val_multi_class_loss: 544.8552 - val_binary_class_loss: 352.9490 - val_multi_class_accuracy: 0.2408 - val_binary_class_accuracy: 0.5009
Epoch 2/10
782/782 [==============================] - 134s 172ms/step - loss: 2.1995 - multi_class_loss: 1.4538 - binary_class_loss: 0.3650 - multi_class_accuracy: 0.4728 - binary_class_accuracy: 0.8423 - val_loss: 471.1795 - val_multi_class_loss: 311.2598 - val_binary_class_loss: 159.5537 - val_multi_class_accuracy: 0.2826 - val_binary_class_accuracy: 0.5098
Epoch 3/10
782/782 [==============================] - 134s 171ms/step - loss: 2.1608 - multi_class_loss: 1.4358 - binary_class_loss: 0.3625 - multi_class_accuracy: 0.4813 - binary_class_accuracy: 0.8426 - val_loss: 696.6667 - val_multi_class_loss: 504.5016 - val_binary_class_

In [ ]:
loss, multi_class_loss, binary_class_loss, multi_class_accuracy, binary_class_accuracy = model.evaluate(x_test,
                                                                                                        {'multi_class': y_test_multiclass,
                                                                                                         'binary_class': y_test_binaryclass})

313/313 [==============================] - 7s 22ms/step - loss: 403.8909 - multi_class_loss: 313.7733 - binary_class_loss: 89.7802 - multi_class_accuracy: 0.3780 - binary_class_accuracy: 0.6729
